In [1]:
import sys, os
sys.path.append(os.getcwd())
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from trace_anal import HarmonicTrace
from tqdm import tqdm
from scipy.signal import savgol_filter
import glob
import h5py as h5
from itertools import product
from matplotlib.patches import Rectangle
from h5object import h5object
from scan_anal import Scan

In [2]:
def Adam_calib(pix_no, trans_pos):
    p = np.arange(pix_no)
    S = trans_pos
    P0 = -0.607656359704894
    gamma = 0.0126547547726546
    S0 = 30000
    X0 = 80.7242798185754
    Gamma = 0.0231429412719341
    g = 120
    alpha = np.deg2rad(86.9625004329325)
    L = 234.351550449396
    theta = np.deg2rad(-1.06891224023216)
    m = 1 #diffraction order
    P_det = P0 + gamma*(S-S0) +p
    X_det = X0-Gamma*P_det 
    beta_dash = np.arctan(X_det*np.cos(theta)/(L + X_det*np.sin(theta)))
    beta = np.deg2rad(90) - beta_dash
    wl = (np.sin(alpha) - np.sin(beta))/(m*g)
    return (wl*1e-3)*1e9



In [3]:
data_folder = '/Volumes/KasiaDrive/Data/Artemis/AndorScan-021-20220624-1310 - MgO 200mm Rotation Intensity'

In [4]:
h5list = os.listdir(data_folder)
h5_file = h5list[1]

In [5]:
test = h5object(data_folder, h5_file)


In [6]:
test.load_stages()

In [7]:
test.pos_df['sample rotation'].unique()

array([  0.,   3.,   6.,   9.,  12.,  15.,  18.,  21.,  24.,  27.,  30.,
        33.,  36.,  39.,  42.,  45.,  48.,  51.,  54.,  57.,  60.,  63.,
        66.,  69.,  72.,  75.,  78.,  81.,  84.,  87.,  90.,  93.,  96.,
        99., 102., 105., 108., 111., 114., 117., 120., 123., 126., 129.,
       132., 135., 138., 141., 144., 147., 150., 153., 156., 159., 162.,
       165., 168., 171., 174., 177., 180.])

In [8]:
test.find_mask('nir waveplate',40)
test.load_data()

/Users/kasia/Documents/Python/Harmonics_Analysis/h5object.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.pos_df['data'][self.masked_idx] = list(self.data)
/Users/kasia/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
test.find_mask('nir waveplate',(55,65))
test.load_data()

In [10]:
test.pos_df

,nir waveplate,sample rotation,data
0,40.0,0.0,"[[110, 107, 105, 101, 107, 101, 107, 110, 105,..."
1,40.0,3.0,"[[117, 101, 103, 107, 106, 103, 107, 105, 107,..."
2,40.0,6.0,"[[99, 106, 107, 108, 107, 108, 104, 110, 108, ..."
3,40.0,9.0,"[[104, 108, 109, 108, 105, 107, 104, 104, 106,..."
4,40.0,12.0,"[[106, 111, 101, 100, 105, 106, 106, 112, 107,..."
...,...,...,...
422,70.0,168.0,None
423,70.0,171.0,None
424,70.0,174.0,None
425,70.0,177.0,None


In [11]:
len(test.pos_df.index)

427

In [12]:
scan = Scan()
scan.set_folder(data_folder)
# fill out which are available from 
scan.set_params(iris=84, wedge=0, rotation=None, MCPPos=70000) # here fill out the variables that you are not scanning through. This scan is Rotation and iris so I fill out the other MCPPos and Wedge
scan.set_verlim(0, -1) #this is the vertical cropping you want to do
scan.set_eVlim((3,40)) #this is the energy limits you want to look at




In [13]:
scan.populate_scan_h5object(test, Adam_calib)

100%|██████████| 244/244 [00:06<00:00, 39.76it/s]


In [14]:
test.pos_df

,nir waveplate,sample rotation,data
0,40.0,0.0,"[[110, 107, 105, 101, 107, 101, 107, 110, 105,..."
1,40.0,3.0,"[[117, 101, 103, 107, 106, 103, 107, 105, 107,..."
2,40.0,6.0,"[[99, 106, 107, 108, 107, 108, 104, 110, 108, ..."
3,40.0,9.0,"[[104, 108, 109, 108, 105, 107, 104, 104, 106,..."
4,40.0,12.0,"[[106, 111, 101, 100, 105, 106, 106, 112, 107,..."
...,...,...,...
422,70.0,168.0,None
423,70.0,171.0,None
424,70.0,174.0,None
425,70.0,177.0,None


In [20]:
plt.imshow(scan.scan_data.Data[0].data

array([[110, 107, 105, ..., 105, 104,  99],
       [101, 102, 106, ..., 116, 107, 106],
       [104, 105, 106, ..., 108, 101, 111],
       ...,
       [101,  98, 103, ..., 103, 102,  99],
       [103, 101, 100, ..., 104, 103, 102],
       [102, 106, 104, ..., 102, 100, 102]], dtype=uint64)

In [16]:
nonadf['nir waveplate'][60]

40.0

In [17]:
for i in nonadf.index:
    print(nonadf['nir waveplate'][i])

40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
